In [ ]:
import re
tfile = "Potârcă_JournMarFam_2015_P9O7.tei.xml"

In [ ]:
s = re.split("_|\.",tfile)
print(s[3])

P9O7


In [ ]:
#@title Mount Google Drive

from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


In [ ]:
# get the weights and model
!cp -r /content/gdrive/MyDrive/Marsview_MRC/HLTC-MRQA/ /content

# Step 1: THE RETRIVER

In [ ]:
# Let's start by installing Haystack

# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install grpcio-tools==1.34.1
!pip install git+https://github.com/deepset-ai/haystack.git
!wget --no-check-certificate https://dl.xpdfreader.com/xpdf-tools-linux-4.03.tar.gz
!tar -xvf xpdf-tools-linux-4.03.tar.gz && sudo cp xpdf-tools-linux-4.03/bin64/pdftotext /usr/local/bin

     |████████████████████████████████| 2.5MB 6.6MB/s 
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-pp48r1pa
  Running command git clone -q https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-pp48r1pa
     |████████████████████████████████| 215kB 7.0MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
     |████████████████████████████████| 81kB 5.3MB/s 
     |████████████████████████████████| 327kB 8.9MB/s 
     |████████████████████████████████| 337kB 10.8MB/s 
     |████████████████████████████████| 92kB 7.6MB/s 
     |████████████████████████████████| 983kB 11.1MB/s 
     |████████████████████████████████| 5.6MB 22.5MB/s 
     |████████████████████████████████| 102kB 10.3MB/s 
     |████████████████████████████████| 8.4MB 34.8MB/s 
     |████████████████████████████████| 348kB 35.5MB/s 
     |████████████████████████████████| 61kB 3.2MB/s 
     |████████████████████████████████| 51k

In [ ]:
pip install jsonlines

In [ ]:
# Here are the imports we need

from haystack.file_converter.txt import TextConverter
from haystack.file_converter.pdf import PDFToTextConverter
from haystack.file_converter.docx import DocxToTextConverter

from haystack.preprocessor.preprocessor import PreProcessor
from haystack import Finder
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

from typing import List
import requests
import pandas as pd
from haystack import Document
from haystack.document_store.faiss import FAISSDocumentStore
from haystack.generator.transformers import RAGenerator
from haystack.retriever.dense import DensePassageRetriever


from typing import Any, Dict, List, Optional

from haystack import Document

import regex as re

06/22/2021 08:19:11 - INFO - faiss.loader -   Loading faiss with AVX2 support.
06/22/2021 08:19:11 - INFO - faiss.loader -   Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
06/22/2021 08:19:11 - INFO - faiss.loader -   Loading faiss.
06/22/2021 08:19:11 - INFO - faiss.loader -   Successfully loaded faiss.
06/22/2021 08:19:11 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [ ]:
# Recommended: Start Elasticsearch using Docker
! docker run -d -p 9200:9200 -e "discovery.type=single-node" elasticsearch:7.9.2

In [ ]:
# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [ ]:
# Connect to Elasticsearch

from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

06/22/2021 08:19:54 - INFO - elasticsearch -   HEAD http://localhost:9200/ [status:200 request:0.106s]
06/22/2021 08:19:55 - INFO - elasticsearch -   PUT http://localhost:9200/document [status:200 request:0.543s]
06/22/2021 08:19:55 - INFO - elasticsearch -   PUT http://localhost:9200/label [status:200 request:0.254s]


In [ ]:
!mkdir /content/data/

In [ ]:
## Final method for splitting.

all_docs = convert_files_to_dicts(dir_path="/content/data")
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=150,
    split_respect_sentence_boundary=True
)
nested_docs = [preprocessor.process(d) for d in all_docs]
docs = [d for x in nested_docs for d in x]

print(f"n_files_input: {len(all_docs)}\nn_docs_output: {len(docs)}")

06/22/2021 08:32:21 - INFO - haystack.preprocessor.utils -   Converting /content/data/m4.txt


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
n_files_input: 1
n_docs_output: 47


In [ ]:
document_store.delete_all_documents()
document_store.write_documents(docs)

06/22/2021 08:32:25 - WARNING - haystack.document_store.elasticsearch -   DEPRECATION WARNINGS: 
                1. delete_all_documents() method is deprecated, please use delete_documents method
                For more details, please refer to the issue: https://github.com/deepset-ai/haystack/issues/1045
                
06/22/2021 08:32:26 - INFO - elasticsearch -   POST http://localhost:9200/document/_delete_by_query [status:200 request:0.129s]
06/22/2021 08:32:29 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.217s]


In [ ]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [ ]:
QUESTIONS = [
 "Who was the winner of the 50$ gift card?",
 "What reason Esther Yoon gave for partnering with otter Ai?",
 "Who was moderating the Q&A panel?",
 "Who is Esther Yoon?",
 "Where one can find the Zoom Topia videos?",
 "What is Zoom Topia video?",
 "What was the value of gift card?"
]

In [ ]:
in_reader = []

for question in QUESTIONS:
  a = retriever.retrieve(query=question, top_k = 3)

  flat_docs_dict: Dict[str, Any] = {}
  for document in a:
      for k, v in document.__dict__.items():
        # print(v)
          if k not in flat_docs_dict:
              flat_docs_dict[k] = []
          flat_docs_dict[k].append(v)

  in_context = ' '.join(flat_docs_dict['text'])

  # Disct for Xlnet input
  x_in =  {'context': 'context input', 'qas': [{'question': "question", 'answers': [], 'qid': 'update', 'question_tokens': [], 'is_impossible': False, 'detected_answers': [{'text': '', 'token_spans': [], 'char_spans': []}]}]}
  #get the detials from the customer
  x_in['context'] = in_context
  for s in x_in['qas']:
    s['qid'] = 'customer_1'
    s['question'] = question
  
  in_reader.append(x_in)


06/22/2021 08:32:31 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.082s]
06/22/2021 08:32:31 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.014s]
06/22/2021 08:32:31 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.010s]
06/22/2021 08:32:31 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.011s]
06/22/2021 08:32:31 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.015s]
06/22/2021 08:32:31 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.011s]
06/22/2021 08:32:31 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.010s]


## Extarct New question on same doc by Ak

In [ ]:
QUESTIONS = [
 "Do you want to say something about the Bruce Bulger campaign? And what's going on there?",
 "How much is too much noise?",
 "Sg&a is a percent of total expense and hcv a what does that mean?",
 "What do we know about net debt market cap?",
 "So Kim, do you think that many companies have zero revenues or we just miss you towing?",
 "How can you have negative Revenue?",
 "How do I get out of here?",
 "Can zoom be integrated with Google meet?",
 "Can zoom know if Raj is speaking?",
 "Will Product Manager takes care of sales?"
]

In [ ]:
QUESTIONS = [
 "Who was the winner of the 50$ gift card?"
]

In [ ]:
for question in QUESTIONS:
  a = retriever.retrieve(query=question, top_k = 5)

06/21/2021 15:53:21 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.022s]


In [ ]:
a

[{'text': "It is all very simple and very quick and efficient. And yeah. Yeah it all just works with The key to zoom, right and I think you guys continue surprising us with new features and functions and that versatility of it just works has been maintained. So it was really great to see it was great. When you to join us after Bobby. - (33:33)  SPEAKER: Bobby Swartz\nYou probably have somebody that's going to get a surprise surprise. Yes. Thank you. Everyone for jumping in. This was really good to get to know a little bit more on the backside of Zoom for those who are able to attend. The show so thank you for coming out. We do have a winner of a $50 gift card today. It is Lashawn Jackson. Thank you very much. Mark.", 'score': 15.737869, 'probability': 0.8773136667338791, 'question': None, 'meta': {'_split_id': 45, 'name': 'm4.txt'}, 'embedding': None, 'id': '1547dbcb9d7bbac22e907a8f7a68bccb'},
 {'text': "They are called meat and they announced the neat bar and the neat board which is a

In [ ]:
in_reader = []

for question in QUESTIONS:
  a = retriever.retrieve(query=question, top_k = 3)

  flat_docs_dict: Dict[str, Any] = {}
  for document in a:
      for k, v in document.__dict__.items():
        # print(v)
          if k not in flat_docs_dict:
              flat_docs_dict[k] = []
          flat_docs_dict[k].append(v)

  in_context = ' '.join(flat_docs_dict['text'])

  # Disct for Xlnet input
  x_in =  {'context': 'context input', 'qas': [{'question': "question", 'answers': [], 'qid': 'update', 'question_tokens': [], 'is_impossible': False, 'detected_answers': [{'text': '', 'token_spans': [], 'char_spans': []}]}]}
  #get the detials from the customer
  x_in['context'] = in_context
  for s in x_in['qas']:
    s['qid'] = 'customer_1'
    s['question'] = question
  
  in_reader.append(x_in)


06/22/2021 08:34:26 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.020s]
06/22/2021 08:34:26 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.010s]
06/22/2021 08:34:26 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.011s]
06/22/2021 08:34:26 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.011s]
06/22/2021 08:34:26 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.011s]
06/22/2021 08:34:26 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.009s]
06/22/2021 08:34:26 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.009s]
06/22/2021 08:34:27 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.009s]
06/22/2021 08:34:27 - INFO - elasticsearch -   P

### No use

In [ ]:
a = retriever.retrieve(query="From where Richard Branson was supposed to call?", top_k = 3)

06/17/2021 12:09:48 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.013s]


In [ ]:
# Parsing the retriever output into a python dict object
flat_docs_dict: Dict[str, Any] = {}
for document in a:
    for k, v in document.__dict__.items():
      # print(v)
        if k not in flat_docs_dict:
            flat_docs_dict[k] = []
        flat_docs_dict[k].append(v)

In [ ]:
# Joining the top_k outputs
in_context = ' '.join(flat_docs_dict['text'])

In [ ]:
in_context

"Yep that and we Richard Branson he was supposed to be calling in from Necker Island, which would have been I would have loved that to show like no fan with like even compromise bandwidth and we can have a crystal-clear call. But yeah, you know, we truly wanted to show the power of video at Zoom Topia. - (23:53)  SPEAKER: Melissa Dillman\nYeah, it was very impressive. Okay. So those were my two Rockstar kind of moments, but let me get to what what all of our guests want to know. Peoples are maybe they are but people still use them all the time and there are scenarios, especially in the modern Workforce where you're like Hey, we're talking about a project. I want to share this with you, but I can't because I'm on a phone call. Let me describe you the content on my screen. It's super weird. And so what we are looking at is okay. So collaboration and communication should be seamless, even when the needs for collaboration communication and change in real time. So I'm sending out this. We s

In [ ]:
# Disct for Xlnet input
x_in =  {'context': 'Want love.', 'qas': [{'question': "What did Luther do at the end of his speech?", 'answers': [], 'qid': '8452474afb0a40f1904a5571b001b6d1', 'question_tokens': [], 'is_impossible': False, 'detected_answers': [{'text': '', 'token_spans': [], 'char_spans': []}]}]}

In [ ]:
#get the detials from the customer
x_in['context'] = in_context
for s in x_in['qas']:
  s['qid'] = 'customer_1'
  s['question'] = "From where Richard Branson was supposed to call?"

In [ ]:
x_in

{'context': "Yep that and we Richard Branson he was supposed to be calling in from Necker Island, which would have been I would have loved that to show like no fan with like even compromise bandwidth and we can have a crystal-clear call. But yeah, you know, we truly wanted to show the power of video at Zoom Topia. - (23:53)  SPEAKER: Melissa Dillman\nYeah, it was very impressive. Okay. So those were my two Rockstar kind of moments, but let me get to what what all of our guests want to know. Peoples are maybe they are but people still use them all the time and there are scenarios, especially in the modern Workforce where you're like Hey, we're talking about a project. I want to share this with you, but I can't because I'm on a phone call. Let me describe you the content on my screen. It's super weird. And so what we are looking at is okay. So collaboration and communication should be seamless, even when the needs for collaboration communication and change in real time. So I'm sending ou

In [ ]:
# store the input file
# !mkdir /content/data
!mkdir /content/data/dev

In [ ]:
# last worked Formate the input like this

import jsonlines
with jsonlines.open('/content/data/dev/DROP-piece.jsonl', mode='w') as writer:
    writer.write({'header': {'dataset': 'RACE', 'mrqa_split': 'dev'}})
    writer.write(x_in)
    writer.close()

### till here

In [ ]:
{"context": "- (33:33)  SPEAKER: Bobby Swartz\nYou probably have somebody that's going to get a surprise surprise. Yes. Thank you. Everyone for jumping in. This was really good to get to know a little bit more on the backside of Zoom for those who are able to attend. The show so thank you for coming out. We do have a winner of a $50 gift card today. It is Lashawn Jackson. Thank you very much. Mark. Decker will be reaching out to you to go ahead and provide that everyone attendees. So think just larger deployments. This is really where the value of appliances come come in. So like I mentioned you get everything that you love with zoom rooms, you get the DIY aspect for those who are cost conscious you have the customization aspect for those who are extra creative and want more of a out-of-this-world zoom room setup. You have the off-the-shelf use case where you can pick and choose the vendors of your choice, right? We have one same thing dedicated mobile cart because when all of our rooms get busy suddenly somebody is dragging the card round and we do use on Wi-Fi because I know Melissa and I have personally wheeled into our warehouse to have some impromptu meetings, but things are getting a little busy in the office and it's a great use case. - (32:43)  SPEAKER: Melissa Dillman\nIt works flawlessly. It really does. We also have one of our one of our friends has put together a challenge our sales Executives. And that is to video record and broadcast.", "qas": [{"question": "Who was the winner of the 50$ gift card?", "answers": [], "qid": "customer_11", "question_tokens": [], "is_impossible": false, "detected_answers": [{"text": "", "token_spans": [], "char_spans": []}]}]}


In [ ]:
!mkdir /content/data/dev

In [ ]:
import jsonlines
with jsonlines.open('/content/data/dev/DROP-piece_bad.jsonl', mode='w') as writer:
    writer.write({'header': {'dataset': 'MARSVIEW', 'mrqa_split': 'dev'}})
    for item in in_reader:
      writer.write(item)
    writer.close()

### Test for top k answers

In [ ]:
pip install jsonlines

In [ ]:
with open('/content/m4.txt', 'r') as file:
    data = file.read()


In [ ]:
data

"- (00:05)  SPEAKER: Bobby Swartz\nHello. Hello, thank you. All of our attendees as we left this room populate here. I'll just keep on monologuing a little bit. Welcome everybody to the November Edition, which is actually here in October of these staring Town Hall. So I just got back from a whirlwind trip to his little tiny show called zootopia and it was I was just commenting before the show starts. I still feel a little tired because it was a whirlwind of fun talking. And customers talking to our partners talking to new partners talking to our vendor Partners talking to our friends and zoom just filled with tons of great conversation a lot of excitement a lot of new features coming to the zoom room capabilities and zoom platform is a whole and I wanted to thank we've got Esther Yoon who is the product marketing manager for conference room Solutions joining us today coming off the heels of zootopia as well, which is kind of a big deal in the zoom World shuts down, San Jose. Pretty goo

In [ ]:
# last worked Formate the input like this

import jsonlines
with jsonlines.open('/content/data/dev/DROP-piece.jsonl', mode='w') as writer:
    writer.write({'header': {'dataset': 'MARSVIEW', 'mrqa_split': 'dev'}})
    writer.write({"context": data, "qas": [{"question": "Sg&a is a percent of total expense and hcv a what does that mean?", "answers": [], "qid": "customer_13", "question_tokens": [], "is_impossible": False, "detected_answers": [{"text": "", "token_spans": [], "char_spans": []}]}]})
    writer.close()

# Step 2 THE READER

In [ ]:
cd /content/HLTC-MRQA

/content/HLTC-MRQA


In [ ]:
pip install -r requirements.txt

     |████████████████████████████████| 411.0MB 40kB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 3.8MB 31.0MB/s 
     |████████████████████████████████| 512kB 31.6MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7557 sha256=875d53d669cb87b2d8489ed3be8ad5b338a743da23a859b030bdf76d1c277cb1
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.5.0 has requirement gast==0.4.0, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement tensorboard~=2.5, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement tensorflow-estimator<2.6.0,>=2.5.0rc0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing ins

In [ ]:
import tensorflow as tf
print(tf.__version__)

1.15.2


In [ ]:
cd /content/HLTC-MRQA

/content/HLTC-MRQA


In [ ]:
# coding=utf-8


####### To get top 5 predicton change the code in the ===== n_best_size
########     To change the output answer length ============  max_answer_length


from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from absl import flags
import absl.logging as _logging  # pylint: disable=unused-import

import collections
import os
import time
import math
import json
import six
import random
import gc
import sys

import numpy as np

if six.PY2:
  import cPickle as pickle
else:
  import pickle
import jsonlines

import tensorflow as tf
import sentencepiece as spm
from src.prepro_utils import preprocess_text, encode_ids, encode_pieces, printable_text
import src.function_builder as function_builder
import src.model_utils as model_utils
import src.mrqa_utils
from src.data_utils import SEP_ID, CLS_ID, VOCAB_SIZE

# Psudo flag then delete it and then create all req flags
# tf.app.flags.DEFINE_string('f', '', 'kernel')
FLAGS = tf.app.flags.FLAGS

# def del_all_flags(FLAGS):
flags_dict = FLAGS._flags()
keys_list = [keys for keys in flags_dict]
for keys in keys_list:
  FLAGS.__delattr__(keys)

# FLAGS = tf.app.flags.FLAGS

SPIECE_UNDERLINE = u'▁'

SEG_ID_P   = 0
SEG_ID_Q   = 1
SEG_ID_CLS = 2
SEG_ID_PAD = 3

# Preprocessing
flags.DEFINE_bool("do_prepro", default=False,
      help="Perform preprocessing only.")
flags.DEFINE_integer("num_proc", default=16,
      help="Number of preprocessing processes.")
flags.DEFINE_integer("proc_id", default=0,
      help="Process id for preprocessing.")

# Model
flags.DEFINE_string("model_config_path", default="/content/gdrive/MyDrive/HLTC-MRQA/model-weights/xlnet_config.json",
      help="Model config path.")
flags.DEFINE_float("dropout", default=0.1,
      help="Dropout rate.")
flags.DEFINE_float("dropatt", default=0.1,
      help="Attention dropout rate.")
flags.DEFINE_integer("clamp_len", default=-1,
      help="Clamp length.")
flags.DEFINE_string("summary_type", default="last",
      help="Method used to summarize a sequence into a vector.")
flags.DEFINE_bool("use_bfloat16", default=False,
      help="Whether to use bfloat16.")

# Parameter initialization
flags.DEFINE_enum("init", default="normal",
                  enum_values=["normal", "uniform"],
                  help="Initialization method.")
flags.DEFINE_float("init_std", default=0.02,
                   help="Initialization std when init is normal.")
flags.DEFINE_float("init_range", default=0.1,
                   help="Initialization std when init is uniform.")

# I/O paths
flags.DEFINE_bool("overwrite_data", default=False,
                  help="If False, will use cached data if available.")
flags.DEFINE_string("init_checkpoint", default="/content/gdrive/MyDrive/HLTC-MRQA/model-weights/model.ckpt-15000",
                    help="checkpoint path for initializing the model. "
                    "Could be a pretrained model or a finetuned model.")
flags.DEFINE_bool("init_global_vars", default=False,
                  help="If true, init all global vars. If false, init "
                  "trainable vars only.")
flags.DEFINE_string("output_dir", default="",
                    help="Output dir for TF records.")
flags.DEFINE_string("predict_dir", default="",
                    help="Dir for predictions.")
flags.DEFINE_string("spiece_model_file", default="/content/gdrive/MyDrive/HLTC-MRQA/spiece.model",
                    help="Sentence Piece model path.")
flags.DEFINE_string("model_dir", default="experiment/mrqa",
                    help="Directory for saving the finetuned model.")
flags.DEFINE_string("train_dir", default="",
                    help="Path of train directory.")
flags.DEFINE_string("dev_dir", default="/content/data/dev",
                    help="Path of development directory.")

# Data preprocessing config
flags.DEFINE_integer("max_seq_length",
                     default=512, help="Max sequence length")
flags.DEFINE_integer("max_query_length",
                     default=128, help="Max query length")
flags.DEFINE_integer("doc_stride",
                     default=128, help="Doc stride")
flags.DEFINE_integer("max_answer_length",
                     default=128, help="Max answer length")                      ########################### MAX ANSWER LENGTH ###########################
flags.DEFINE_bool("uncased", default=False, help="Use uncased data.")

# TPUs and machines
flags.DEFINE_bool("use_tpu", default=False, help="whether to use TPU.")
flags.DEFINE_integer("num_hosts", default=1, help="How many TPU hosts.")
flags.DEFINE_integer("num_core_per_host", default=8,
      help="8 for TPU v2 and v3-8, 16 for larger TPU v3 pod. In the context "
      "of GPU training, it refers to the number of GPUs used.")
flags.DEFINE_string("tpu_job_name", default=None, help="TPU worker job name.")
flags.DEFINE_string("tpu", default=None, help="TPU name.")
flags.DEFINE_string("tpu_zone", default=None, help="TPU zone.")
flags.DEFINE_string("gcp_project", default=None, help="gcp project.")
flags.DEFINE_string("master", default=None, help="master")
flags.DEFINE_integer("iterations", default=1000,
                     help="number of iterations per TPU training loop.")

# Training
flags.DEFINE_bool("do_train", default=True, help="whether to do training")
flags.DEFINE_integer("train_batch_size", default=48,
                     help="batch size for training")
flags.DEFINE_integer("train_steps", default=8000,
                     help="Number of training steps")
flags.DEFINE_integer("warmup_steps", default=0, help="number of warmup steps")
flags.DEFINE_integer("save_steps", default=None,
                     help="Save the model for every save_steps. "
                     "If None, not to save any model.")
flags.DEFINE_integer("max_save", default=0,
                     help="Max number of checkpoints to save. "
                     "Use 0 to save all.")
flags.DEFINE_integer("shuffle_buffer", default=2048,
                     help="Buffer size used for shuffle.")

# Optimization
flags.DEFINE_float("learning_rate", default=3e-5, help="initial learning rate")
flags.DEFINE_float("min_lr_ratio", default=0.0,
                   help="min lr ratio for cos decay.")
flags.DEFINE_float("clip", default=1.0, help="Gradient clipping")
flags.DEFINE_float("weight_decay", default=0.00, help="Weight decay rate")
flags.DEFINE_float("adam_epsilon", default=1e-6, help="Adam epsilon")
flags.DEFINE_string("decay_method", default="poly", help="poly or cos")
flags.DEFINE_float("lr_layer_decay_rate", default=0.75,
                   help="Top layer: lr[L] = FLAGS.learning_rate."
                   "Lower layers: lr[l-1] = lr[l] * lr_layer_decay_rate.")

# Eval / Prediction
flags.DEFINE_bool("do_predict", default=False, help="whether to do predict")
flags.DEFINE_integer("predict_batch_size", default=32,
                     help="batch size for prediction")
flags.DEFINE_integer("n_best_size", default=5,
                     help="n best size for predictions")
flags.DEFINE_integer("start_n_top", default=5, help="Beam size for span start.")
flags.DEFINE_integer("end_n_top", default=5, help="Beam size for span end.")
flags.DEFINE_string("target_eval_key", default="best_f1",
                    help="Use has_ans_f1 for Model I.")
flags.DEFINE_string("export_dir_base", default="/content/gdrive/MyDrive/HLTC-MRQA/exported-tf-model-1.15.2",
                    help="Path of exported models.")


# def del_all_flags(FLAGS):
#   flags_dict = FLAGS._flags()
#   keys_list = [keys for keys in flags_dict]
#   for keys in keys_list:
#     FLAGS.delattr(keys)

# FLAGS = flags.FLAGS
FLAGS = tf.app.flags.FLAGS


class MRQAExample(object):
  """A single training/test example for simple sequence classification.

     For examples without an answer, the start and end position are -1.
  """

  def __init__(self,
               qas_id,
               question_text,
               paragraph_text,
               orig_answer_text=None,
               start_position=None,
               send_position=None,
               is_impossible=False):
    self.qas_id = qas_id
    self.question_text = question_text
    self.paragraph_text = paragraph_text
    self.orig_answer_text = orig_answer_text
    self.start_position = start_position
    self.send_position = send_position
    self.is_impossible = is_impossible

  def __str__(self):
    return self.__repr__()

  def __repr__(self):
    s = ""
    s += "qas_id: %s" % (printable_text(self.qas_id))
    s += ", question_text: %s" % (
        printable_text(self.question_text))
    s += ", paragraph_text: [%s]" % (" ".join(self.paragraph_text))
    if self.start_position:
      s += ", start_position: %d" % (self.start_position)
    if self.start_position:
      s += ", is_impossible: %r" % (self.is_impossible)
    if self.start_position:
      s += ", send_position: %d" % (self.send_position)
    return s

class InputFeatures(object):
  """A single set of features of data."""

  def __init__(self,
               unique_id,
               example_index,
               doc_span_index,
               tok_start_to_orig_index,
               tok_end_to_orig_index,
               token_is_max_context,
               input_ids,
               input_mask,
               p_mask,
               segment_ids,
               paragraph_len,
               cls_index,
               start_position=None,
               end_position=None,
               is_impossible=None):
    self.unique_id = unique_id
    self.example_index = example_index
    self.doc_span_index = doc_span_index
    self.tok_start_to_orig_index = tok_start_to_orig_index # paragraph
    self.tok_end_to_orig_index = tok_end_to_orig_index     # paragraph
    self.token_is_max_context = token_is_max_context
    self.input_ids = input_ids                             # context+question
    self.input_mask = input_mask
    self.p_mask = p_mask
    self.segment_ids = segment_ids
    self.paragraph_len = paragraph_len
    self.cls_index = cls_index
    self.start_position = start_position
    self.end_position = end_position
    self.is_impossible = is_impossible
#########################################################################################Read Test data######################################################################################
######################################################################################Read Test data###############################################################################################
####################################################################################Read Test data#####################################################################################################
def read_mrqa_data(input_file, is_training):
  """Read a QA data jsonl file into a list of Examples."""
  with tf.gfile.Open(input_file, "r") as reader:
    input_data = jsonlines.Reader(reader)

  examples = []
  for entry in input_data:
    if u'header' in entry:
      continue

    assert type(entry) == dict
    assert u'context' in entry
    assert u'qas' in entry

    paragraph_text = entry["context"]

    for qa in entry["qas"]:
      assert u'qid' in qa
      assert u'question' in qa
      assert u'detected_answers' in qa
      assert u'text' in qa[u'detected_answers'][0]
      assert u'char_spans' in qa[u'detected_answers'][0]

      qas_id = qa["qid"]
      question_text = qa["question"]
      start_position = None
      send_position = None
      orig_answer_text = None
      is_impossible = False
      
      if is_training:
        is_impossible = False if qa["detected_answers"]!=[] else True
        # if (len(qa["answers"]) != 1) and (not is_impossible):  # TriviaQA may have several answers, choose the first one
        #     raise ValueError(
        #         "For training, each question should have exactly 1 answer.")
        if not is_impossible:
          answer = qa["detected_answers"][0]
          orig_answer_text = answer["text"]
          start_position = answer["char_spans"][0][0]
          send_position = answer["char_spans"][0][1]
          assert type(start_position)==int
        else:
          print(qa["answers"])
          print("="*80)
          start_position = -1
          send_position = -1
          orig_answer_text = ""        

      example = MRQAExample(
          qas_id=qas_id,
          question_text=question_text,
          paragraph_text=paragraph_text,
          orig_answer_text=orig_answer_text,
          start_position=start_position,
          send_position=send_position,
          is_impossible=is_impossible)
      examples.append(example)
  return examples

def arrange_mrqa_data(input_data, is_training):
  """Read a QA data jsonl file into a list of Examples."""
  examples = []
  entry = input_data

  assert type(entry) == dict
  assert u'context' in entry
  assert u'qas' in entry

  paragraph_text = entry["context"]
  print(input_data)
  print("="*80)
  print(len(entry["qas"]))
  for qa in entry["qas"]:
    assert u'qid' in qa
    assert u'question' in qa
    assert u'detected_answers' in qa
    assert u'text' in qa[u'detected_answers'][0]
    assert u'char_spans' in qa[u'detected_answers'][0]

    qas_id = qa["qid"]
    question_text = qa["question"]
    start_position = None
    send_position = None
    orig_answer_text = None
    is_impossible = False
    
    if is_training:
      is_impossible = False if qa["detected_answers"]!=[] else True
      # if (len(qa["answers"]) != 1) and (not is_impossible):  # TriviaQA may have several answers, choose the first one
      #     raise ValueError(
      #         "For training, each question should have exactly 1 answer.")
      if not is_impossible:
        answer = qa["detected_answers"][0]
        orig_answer_text = answer["text"]
        start_position = answer["char_spans"][0][0]
        send_position = answer["char_spans"][0][1]
        assert type(start_position)==int
      else:
        start_position = -1
        send_position = -1
        orig_answer_text = ""        

    example = MRQAExample(
        qas_id=qas_id,
        question_text=question_text,
        paragraph_text=paragraph_text,
        orig_answer_text=orig_answer_text,
        start_position=start_position,
        send_position=send_position,
        is_impossible=is_impossible)
    examples.append(example)
    print(len(examples))
    print("="*80)
  return examples

def _convert_index(index, pos, M=None, is_start=True):
  if index[pos] is not None:
    return index[pos]
  N = len(index)
  rear = pos
  while rear < N - 1 and index[rear] is None:
    rear += 1
  front = pos
  while front > 0 and index[front] is None:
    front -= 1
  assert index[front] is not None or index[rear] is not None
  if index[front] is None:
    if index[rear] >= 1:
      if is_start:
        return 0
      else:
        return index[rear] - 1
    return index[rear]
  if index[rear] is None:
    if M is not None and index[front] < M - 1:
      if is_start:
        return index[front] + 1
      else:
        return M - 1
    return index[front]
  if is_start:
    if index[rear] > index[front] + 1:
      return index[front] + 1
    else:
      return index[rear]
  else:
    if index[rear] > index[front] + 1:
      return index[rear] - 1
    else:
      return index[front]

def convert_examples_to_features(examples, sp_model, max_seq_length,
                                 doc_stride, max_query_length, is_training,
                                 output_fn):
  """Loads a data file into a list of `InputBatch`s."""

  cnt_pos, cnt_neg = 0, 0
  unique_id = 1000000000
  max_N, max_M = 1024, 1024
  f = np.zeros((max_N, max_M), dtype=np.float32)

  for (example_index, example) in enumerate(examples):

    if example_index % 100 == 0:
      tf.logging.info('Converting {}/{} pos {} neg {}'.format(
          example_index, len(examples), cnt_pos, cnt_neg))

    query_tokens = encode_ids(
        sp_model,
        preprocess_text(example.question_text, lower=FLAGS.uncased))

    if len(query_tokens) > max_query_length:
      query_tokens = query_tokens[0:max_query_length]

    paragraph_text = example.paragraph_text
    para_tokens = encode_pieces(
        sp_model,
        preprocess_text(example.paragraph_text, lower=FLAGS.uncased))

    chartok_to_tok_index = []
    tok_start_to_chartok_index = []
    tok_end_to_chartok_index = []
    char_cnt = 0
    for i, token in enumerate(para_tokens):
      chartok_to_tok_index.extend([i] * len(token))
      tok_start_to_chartok_index.append(char_cnt)
      char_cnt += len(token)
      tok_end_to_chartok_index.append(char_cnt - 1)

    tok_cat_text = ''.join(para_tokens).replace(SPIECE_UNDERLINE, ' ')
    N, M = len(paragraph_text), len(tok_cat_text)

    if N > max_N or M > max_M:
      max_N = max(N, max_N)
      max_M = max(M, max_M)
      f = np.zeros((max_N, max_M), dtype=np.float32)
      gc.collect()

    g = {}

    def _lcs_match(max_dist):
      f.fill(0)
      g.clear()

      ### longest common sub sequence
      # f[i, j] = max(f[i - 1, j], f[i, j - 1], f[i - 1, j - 1] + match(i, j))
      for i in range(N):

        # unlike standard LCS, this is specifically optimized for the setting
        # because the mismatch between sentence pieces and original text will
        # be small
        for j in range(i - max_dist, i + max_dist):
          if j >= M or j < 0: continue

          if i > 0:
            g[(i, j)] = 0
            f[i, j] = f[i - 1, j]

          if j > 0 and f[i, j - 1] > f[i, j]:
            g[(i, j)] = 1
            f[i, j] = f[i, j - 1]

          f_prev = f[i - 1, j - 1] if i > 0 and j > 0 else 0
          if (preprocess_text(paragraph_text[i], lower=FLAGS.uncased,
              remove_space=False)
              == tok_cat_text[j]
              and f_prev + 1 > f[i, j]):
            g[(i, j)] = 2
            f[i, j] = f_prev + 1

    max_dist = abs(N - M) + 5
    for _ in range(2):
      _lcs_match(max_dist)
      if f[N - 1, M - 1] > 0.8 * N: break
      max_dist *= 2

    orig_to_chartok_index = [None] * N
    chartok_to_orig_index = [None] * M
    i, j = N - 1, M - 1
    while i >= 0 and j >= 0:
      if (i, j) not in g: break
      if g[(i, j)] == 2:
        orig_to_chartok_index[i] = j
        chartok_to_orig_index[j] = i
        i, j = i - 1, j - 1
      elif g[(i, j)] == 1:
        j = j - 1
      else:
        i = i - 1

    if all(v is None for v in orig_to_chartok_index) or f[N - 1, M - 1] < 0.8 * N:
      print('MISMATCH DETECTED!')
      continue

    tok_start_to_orig_index = []
    tok_end_to_orig_index = []
    for i in range(len(para_tokens)):
      start_chartok_pos = tok_start_to_chartok_index[i]
      end_chartok_pos = tok_end_to_chartok_index[i]
      start_orig_pos = _convert_index(chartok_to_orig_index, start_chartok_pos,
                                      N, is_start=True)
      end_orig_pos = _convert_index(chartok_to_orig_index, end_chartok_pos,
                                    N, is_start=False)

      tok_start_to_orig_index.append(start_orig_pos)
      tok_end_to_orig_index.append(end_orig_pos)

    if not is_training:
      tok_start_position = tok_end_position = None

    if is_training and example.is_impossible:
      tok_start_position = -1
      tok_end_position = -1

    if is_training and not example.is_impossible:
      start_position = example.start_position
      # end_position = start_position + len(example.orig_answer_text) - 1
      end_position = example.send_position

      start_chartok_pos = _convert_index(orig_to_chartok_index, start_position,
                                         is_start=True)
      tok_start_position = chartok_to_tok_index[start_chartok_pos]

      end_chartok_pos = _convert_index(orig_to_chartok_index, end_position,
                                       is_start=False)
      tok_end_position = chartok_to_tok_index[end_chartok_pos]
      assert tok_start_position <= tok_end_position

    def _piece_to_id(x):
      if six.PY2 and isinstance(x, unicode):
        x = x.encode('utf-8')
      return sp_model.PieceToId(x)

    all_doc_tokens = list(map(_piece_to_id, para_tokens))

    # The -3 accounts for [CLS], [SEP] and [SEP]
    max_tokens_for_doc = max_seq_length - len(query_tokens) - 3

    # We can have documents that are longer than the maximum sequence length.
    # To deal with this we do a sliding window approach, where we take chunks
    # of the up to our max length with a stride of `doc_stride`.
    _DocSpan = collections.namedtuple(  # pylint: disable=invalid-name
        "DocSpan", ["start", "length"])
    doc_spans = []
    start_offset = 0
    while start_offset < len(all_doc_tokens):
      length = len(all_doc_tokens) - start_offset
      if length > max_tokens_for_doc:
        length = max_tokens_for_doc
      doc_spans.append(_DocSpan(start=start_offset, length=length))
      if start_offset + length == len(all_doc_tokens):
        break
      start_offset += min(length, doc_stride)

    for (doc_span_index, doc_span) in enumerate(doc_spans):
      tokens = []
      token_is_max_context = {}
      segment_ids = []
      p_mask = []

      cur_tok_start_to_orig_index = []
      cur_tok_end_to_orig_index = []

      for i in range(doc_span.length):
        split_token_index = doc_span.start + i

        cur_tok_start_to_orig_index.append(
            tok_start_to_orig_index[split_token_index])
        cur_tok_end_to_orig_index.append(
            tok_end_to_orig_index[split_token_index])

        is_max_context = _check_is_max_context(doc_spans, doc_span_index,
                                               split_token_index)
        token_is_max_context[len(tokens)] = is_max_context
        tokens.append(all_doc_tokens[split_token_index])
        segment_ids.append(SEG_ID_P)
        p_mask.append(0)

      paragraph_len = len(tokens)

      tokens.append(SEP_ID)
      segment_ids.append(SEG_ID_P)
      p_mask.append(1)

      # note(zhiliny): we put P before Q
      # because during pretraining, B is always shorter than A
      for token in query_tokens:
        tokens.append(token)
        segment_ids.append(SEG_ID_Q)
        p_mask.append(1)
      tokens.append(SEP_ID)
      segment_ids.append(SEG_ID_Q)
      p_mask.append(1)

      cls_index = len(segment_ids)
      tokens.append(CLS_ID)
      segment_ids.append(SEG_ID_CLS)
      p_mask.append(0)

      input_ids = tokens

      # The mask has 0 for real tokens and 1 for padding tokens. Only real
      # tokens are attended to.
      input_mask = [0] * len(input_ids)

      # Zero-pad up to the sequence length.
      while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(1)
        segment_ids.append(SEG_ID_PAD)
        p_mask.append(1)

      assert len(input_ids) == max_seq_length
      assert len(input_mask) == max_seq_length
      assert len(segment_ids) == max_seq_length
      assert len(p_mask) == max_seq_length

      span_is_impossible = example.is_impossible
      start_position = None
      end_position = None
      if is_training and not span_is_impossible:
        # For training, if our document chunk does not contain an annotation
        # we throw it out, since there is nothing to predict.
        doc_start = doc_span.start
        doc_end = doc_span.start + doc_span.length - 1
        out_of_span = False
        if not (tok_start_position >= doc_start and
                tok_end_position <= doc_end):
          # print("out of span")
          # print("{}|{}|{}|{}".format(doc_start,tok_start_position,tok_end_position,doc_end))
          out_of_span = True
        if out_of_span:
          # continue
          start_position = 0
          end_position = 0
          span_is_impossible = True
        else:
          # note(zhiliny): we put P before Q, so doc_offset should be zero.
          # doc_offset = len(query_tokens) + 2
          doc_offset = 0
          start_position = tok_start_position - doc_start + doc_offset
          end_position = tok_end_position - doc_start + doc_offset

      if is_training and span_is_impossible:
        start_position = cls_index
        end_position = cls_index

          # note(zhiliny): With multi processing,
          # the example_index is actually the index within the current process
          # therefore we use example_index=None to avoid being used in the future.
          # The current code does not use example_index of training data.
      if is_training:
        feat_example_index = None
      else:
        feat_example_index = example_index

      feature = InputFeatures(
          unique_id=unique_id,
          example_index=feat_example_index,
          doc_span_index=doc_span_index,
          tok_start_to_orig_index=cur_tok_start_to_orig_index,
          tok_end_to_orig_index=cur_tok_end_to_orig_index,
          token_is_max_context=token_is_max_context,
          input_ids=input_ids,
          input_mask=input_mask,
          p_mask=p_mask,
          segment_ids=segment_ids,
          paragraph_len=paragraph_len,
          cls_index=cls_index,
          start_position=start_position,
          end_position=end_position,
          is_impossible=span_is_impossible)

      # Run callback
      output_fn(feature)

      unique_id += 1
      if span_is_impossible:
        cnt_neg += 1
      else:
        cnt_pos += 1

  tf.logging.info("Total number of instances: {} = pos {} neg {}".format(
      cnt_pos + cnt_neg, cnt_pos, cnt_neg))
	

def _check_is_max_context(doc_spans, cur_span_index, position):
  """Check if this is the 'max context' doc span for the token."""

  # Because of the sliding window approach taken to scoring documents, a single ######################################################################################################################### 512
  # token can appear in multiple documents. E.g.
  #  Doc: the man went to the store and bought a gallon of milk
  #  Span A: the man went to the
  #  Span B: to the store and bought
  #  Span C: and bought a gallon of
  #  ...
  #
  # Now the word 'bought' will have two scores from spans B and C. We only
  # want to consider the score with "maximum context", which we define as
  # the *minimum* of its left and right context (the *sum* of left and
  # right context will always be the same, of course).
  #
  # In the example the maximum context for 'bought' would be span C since
  # it has 1 left context and 3 right context, while span B has 4 left context
  # and 0 right context.
  best_score = None
  best_span_index = None
  for (span_index, doc_span) in enumerate(doc_spans):
    end = doc_span.start + doc_span.length - 1
    if position < doc_span.start:
      continue
    if position > end:
      continue
    num_left_context = position - doc_span.start
    num_right_context = end - position
    score = min(num_left_context, num_right_context) + 0.01 * doc_span.length
    if best_score is None or score > best_score:
      best_score = score
      best_span_index = span_index

  return cur_span_index == best_span_index

class FeatureWriter(object):
  """Writes InputFeature to TF example file."""

  def __init__(self, is_training):
    self.is_training = is_training
    self.num_features = 0

  def process_feature(self, feature):
    """Write a InputFeature to the TFRecordWriter as a tf.train.Example."""
    self.num_features += 1

    def create_int_feature(values):
      feature = tf.train.Feature(
          int64_list=tf.train.Int64List(value=list(values)))
      return feature

    def create_float_feature(values):
      f = tf.train.Feature(float_list=tf.train.FloatList(value=list(values)))
      return f

    features = collections.OrderedDict()
    features["unique_ids"] = create_int_feature([feature.unique_id])
    features["input_ids"] = create_int_feature(feature.input_ids)
    features["input_mask"] = create_float_feature(feature.input_mask)
    features["p_mask"] = create_float_feature(feature.p_mask)
    features["segment_ids"] = create_int_feature(feature.segment_ids)

    features["cls_index"] = create_int_feature([feature.cls_index])

    if self.is_training:
      features["start_positions"] = create_int_feature([feature.start_position])
      features["end_positions"] = create_int_feature([feature.end_position])
      impossible = 0
      if feature.is_impossible:
        impossible = 1
      features["is_impossible"] = create_float_feature([impossible])

    tf_example = tf.train.Example(features=tf.train.Features(feature=features))
    return tf_example.SerializeToString()

RawResult = collections.namedtuple("RawResult",
    ["unique_id", "start_top_log_probs", "start_top_index",
    "end_top_log_probs", "end_top_index", "cls_logits"])

_PrelimPrediction = collections.namedtuple(  # pylint: disable=invalid-name
    "PrelimPrediction",
    ["feature_index", "start_index", "end_index",
    "start_log_prob", "end_log_prob"])

_NbestPrediction = collections.namedtuple(  # pylint: disable=invalid-name
    "NbestPrediction", ["text", "start_log_prob", "end_log_prob"])
################################################################################## Print Prediction Out ###############################################################################################
################################################################################## Print Prediction Out ###############################################################################################
################################################################################## Print Prediction Out ###############################################################################################
def get_predictions(all_examples, all_features, all_results, n_best_size,
                    max_answer_length, lis):
  """Write final predictions to the json file and log-odds of null if needed."""
  tf.logging.info("Getting predictions")

  example_index_to_features = collections.defaultdict(list)
  for feature in all_features:
    example_index_to_features[feature.example_index].append(feature)

  unique_id_to_result = {}
  for result in all_results:
    unique_id_to_result[result.unique_id] = result

  all_predictions = collections.OrderedDict()

  for (example_index, example) in enumerate(all_examples):
    features = example_index_to_features[example_index]

    prelim_predictions = []
    # keep track of the minimum score of null start+end of position 0
    score_null = 1000000  # large and positive

    for (feature_index, feature) in enumerate(features):
      result = unique_id_to_result[feature.unique_id]

      cur_null_score = result.cls_logits

      # if we could have irrelevant answers, get the min score of irrelevant
      score_null = min(score_null, cur_null_score)

      for i in range(FLAGS.start_n_top):
        for j in range(FLAGS.end_n_top):
          start_log_prob = result.start_top_log_probs[i]
          start_index = result.start_top_index[i]

          j_index = i * FLAGS.end_n_top + j

          end_log_prob = result.end_top_log_probs[j_index]
          end_index = result.end_top_index[j_index]

          # We could hypothetically create invalid predictions, e.g., predict
          # that the start of the span is in the question. We throw out all
          # invalid predictions.
          if start_index >= feature.paragraph_len - 1:
            continue
          if end_index >= feature.paragraph_len - 1:
            continue

          if not feature.token_is_max_context.get(start_index, False):
            continue
          if end_index < start_index:
            continue
          length = end_index - start_index + 1
          if length > max_answer_length:
            continue

          prelim_predictions.append(
              _PrelimPrediction(
                  feature_index=feature_index,
                  start_index=start_index,
                  end_index=end_index,
                  start_log_prob=start_log_prob,
                  end_log_prob=end_log_prob))

    prelim_predictions = sorted(
        prelim_predictions,
        key=lambda x: (x.start_log_prob + x.end_log_prob),
        reverse=True)

    seen_predictions = {}
    nbest = []
    for pred in prelim_predictions:
      if len(nbest) >= n_best_size:
        break
      feature = features[pred.feature_index]

      tok_start_to_orig_index = feature.tok_start_to_orig_index
      tok_end_to_orig_index = feature.tok_end_to_orig_index
      start_orig_pos = tok_start_to_orig_index[pred.start_index]
      end_orig_pos = tok_end_to_orig_index[pred.end_index]

      paragraph_text = example.paragraph_text
      final_text = paragraph_text[start_orig_pos: end_orig_pos + 1].strip()

      if final_text in seen_predictions:
        continue

      seen_predictions[final_text] = True

      nbest.append(
          _NbestPrediction(
              text=final_text,
              start_log_prob=pred.start_log_prob,
              end_log_prob=pred.end_log_prob))
 ################# Add top 5 ####################  
    all_pred = []
    s_t = []
    e_t = []
    for pred in prelim_predictions:
  
      feature = features[pred.feature_index]

      tok_start_to_orig_index = feature.tok_start_to_orig_index
      tok_end_to_orig_index = feature.tok_end_to_orig_index
      start_orig_pos = tok_start_to_orig_index[pred.start_index]
      end_orig_pos = tok_end_to_orig_index[pred.end_index]

      paragraph_text = example.paragraph_text
      final_text = paragraph_text[start_orig_pos: end_orig_pos + 1].strip()

      xx = pred.start_log_prob
      yy = pred.end_log_prob

      j  = xx + yy

      all_pred.append([final_text,j])

      # all_pred.append([pred])
      # all_pred.append(final_text)   ## give all the selected spans
      # all_pred.append([start_orig_pos,end_orig_pos])    ## give start and end token_num for all the selected spans
      s_t.append(start_orig_pos)
      e_t.append(end_orig_pos)
    lis.append(all_pred)

    # all_pred.append([min(s_t),max(e_t)])  ## give min start and max end token_num for all the selected spans to get one span

    # start_orig_pos_min = min(s_t)
    # end_orig_pos_max = max(e_t)
    # paragraph_text = example.paragraph_text  ## give one max span span
    # final_max_text = paragraph_text[start_orig_pos_min: end_orig_pos_max + 1].strip()
    # j = [start_orig_pos_min, end_orig_pos_max]

    # all_pred.append([j,final_max_text,])


################# Add top 5 ####################        
    # In very rare edge cases we could have no valid predictions. So we
    # just create a nonce prediction in this case to avoid failure.
    if not nbest:
      nbest.append(
          _NbestPrediction(text="", start_log_prob=-1e6,
          end_log_prob=-1e6))

    total_scores = []
    best_non_null_entry = None
    for entry in nbest:
      total_scores.append(entry.start_log_prob + entry.end_log_prob)
      if not best_non_null_entry:
        best_non_null_entry = entry

    assert best_non_null_entry is not None

    all_predictions[example.qas_id] = best_non_null_entry.text

  # return all_predictions, all_pred
  return all_predictions, lis

def _get_best_indexes(logits, n_best_size):
  """Get the n-best logits from a list."""
  index_and_score = sorted(enumerate(logits), key=lambda x: x[1], reverse=True)

  best_indexes = []
  for i in range(len(index_and_score)):
    if i >= n_best_size:
      break
    best_indexes.append(index_and_score[i][0])
  return best_indexes


def _compute_softmax(scores):
  """Compute softmax probability over raw logits."""
  if not scores:
    return []

  max_score = None
  for score in scores:
    if max_score is None or score > max_score:
      max_score = score

  exp_scores = []
  total_sum = 0.0
  for score in scores:
    x = math.exp(score - max_score)
    exp_scores.append(x)
    total_sum += x

  probs = []
  for score in exp_scores:
    probs.append(score / total_sum)
  return probs


def input_fn_builder(input_glob, seq_length, is_training, drop_remainder,
                     num_hosts, num_threads=8):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  name_to_features = {
      "unique_ids": tf.FixedLenFeature([], tf.int64),
      "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "input_mask": tf.FixedLenFeature([seq_length], tf.float32),
      "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "cls_index": tf.FixedLenFeature([], tf.int64),
      "p_mask": tf.FixedLenFeature([seq_length], tf.float32)
  }

  if is_training:
    name_to_features["start_positions"] = tf.FixedLenFeature([], tf.int64)
    name_to_features["end_positions"] = tf.FixedLenFeature([], tf.int64)
    name_to_features["is_impossible"] = tf.FixedLenFeature([], tf.float32)

  tf.logging.info("Input tfrecord file glob {}".format(input_glob))
  global_input_paths = tf.gfile.Glob(input_glob)
  tf.logging.info("Find {} input paths {}".format(
      len(global_input_paths), global_input_paths))

  def _decode_record(record, name_to_features):
    """Decodes a record to a TensorFlow example."""
    example = tf.parse_single_example(record, name_to_features)

    # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
    # So cast all int64 to int32.
    for name in list(example.keys()):
      t = example[name]
      if t.dtype == tf.int64:
        t = tf.cast(t, tf.int32)
      example[name] = t

    return example

  def input_fn(params):
    """The actual input function."""
    if FLAGS.use_tpu:
      batch_size = params["batch_size"]
    elif is_training:
      batch_size = FLAGS.train_batch_size
    else:
      batch_size = FLAGS.predict_batch_size

    # Split tfrecords across hosts
    if num_hosts > 1:
      host_id = params["context"].current_host
      num_files = len(global_input_paths)
      if num_files >= num_hosts:
        num_files_per_host = (num_files + num_hosts - 1) // num_hosts
        my_start_file_id = host_id * num_files_per_host
        my_end_file_id = min((host_id + 1) * num_files_per_host, num_files)
        input_paths = global_input_paths[my_start_file_id: my_end_file_id]
      tf.logging.info("Host {} handles {} files".format(host_id,
                                                        len(input_paths)))
    else:
      input_paths = global_input_paths

    if len(input_paths) == 1:
      d = tf.data.TFRecordDataset(input_paths[0])
      # For training, we want a lot of parallel reading and shuffling.
      # For eval, we want no shuffling and parallel reading doesn't matter.
      if is_training:
        d = d.shuffle(buffer_size=FLAGS.shuffle_buffer)
        d = d.repeat()
    else:
      d = tf.data.Dataset.from_tensor_slices(input_paths)
      # file level shuffle
      d = d.shuffle(len(input_paths)).repeat()

      # `cycle_length` is the number of parallel files that get read.
      cycle_length = min(num_threads, len(input_paths))

      d = d.apply(
          tf.contrib.data.parallel_interleave(
              tf.data.TFRecordDataset,
              sloppy=is_training,
              cycle_length=cycle_length))

      if is_training:
        # sample level shuffle
        d = d.shuffle(buffer_size=FLAGS.shuffle_buffer)

    d = d.apply(
        tf.contrib.data.map_and_batch(
            lambda record: _decode_record(record, name_to_features),
            batch_size=batch_size,
            num_parallel_batches=num_threads,
            drop_remainder=drop_remainder))
    d = d.prefetch(1024)

    return d

  return input_fn

def get_model_fn():
  def model_fn(features, labels, mode, params):
    #### Training or Evaluation
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    #### Get loss from inputs
    outputs = function_builder.get_qa_outputs(FLAGS, features, is_training)

    #### Check model parameters
    num_params = sum([np.prod(v.shape) for v in tf.trainable_variables()])
    tf.logging.info('#params: {}'.format(num_params))

    scaffold_fn = None

    #### Evaluation mode
    if mode == tf.estimator.ModeKeys.PREDICT:
      if FLAGS.init_checkpoint:
        tf.logging.info("init_checkpoint not being used in predict mode.")

      predictions = {
          "unique_ids": features["unique_ids"],
          "start_top_index": outputs["start_top_index"],
          "start_top_log_probs": outputs["start_top_log_probs"],
          "end_top_index": outputs["end_top_index"],
          "end_top_log_probs": outputs["end_top_log_probs"],
          "cls_logits": outputs["cls_logits"]
      }

      if FLAGS.use_tpu:
        output_spec = tf.contrib.tpu.TPUEstimatorSpec(
            mode=mode, predictions=predictions, scaffold_fn=scaffold_fn)
      else:
        output_spec = tf.estimator.EstimatorSpec(
            mode=mode, predictions=predictions)
      return output_spec

    ### Compute loss
    seq_length = tf.shape(features["input_ids"])[1]
    def compute_loss(log_probs, positions):
      one_hot_positions = tf.one_hot(positions, depth=seq_length, dtype=tf.float32)

      loss = - tf.reduce_sum(one_hot_positions * log_probs, axis=-1)
      loss = tf.reduce_mean(loss)
      return loss

    start_loss = compute_loss(
        outputs["start_log_probs"], features["start_positions"])
    end_loss = compute_loss(
        outputs["end_log_probs"], features["end_positions"])

    total_loss = (start_loss + end_loss) * 0.5

    cls_logits = outputs["cls_logits"] ## answerability loss
    is_impossible = tf.reshape(features["is_impossible"], [-1])
    regression_loss = tf.nn.sigmoid_cross_entropy_with_logits(
        labels=is_impossible, logits=cls_logits)
    regression_loss = tf.reduce_mean(regression_loss)

    # note(zhiliny): by default multiply the loss by 0.5 so that the scale is
    # comparable to start_loss and end_loss
    total_loss += regression_loss * 0.5

    #### Configuring the optimizer
    train_op, learning_rate, _ = model_utils.get_train_op(FLAGS, total_loss)

    monitor_dict = {}
    monitor_dict["lr"] = learning_rate

    #### load pretrained models
    scaffold_fn = model_utils.init_from_checkpoint(FLAGS)

    #### Constucting training TPUEstimatorSpec with new cache.
    if FLAGS.use_tpu:
      host_call = function_builder.construct_scalar_host_call(
          monitor_dict=monitor_dict,
          model_dir=FLAGS.model_dir,
          prefix="train/",
          reduce_fn=tf.reduce_mean)

      train_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode, loss=total_loss, train_op=train_op, host_call=host_call,
          scaffold_fn=scaffold_fn)
    else:
      train_spec = tf.estimator.EstimatorSpec(
          mode=mode, loss=total_loss, train_op=train_op)

    return train_spec

  return model_fn

def _get_spm_basename():
  spm_basename = os.path.basename(FLAGS.spiece_model_file)
  return spm_basename

################################################################################################# predictor ############################################################################################################################################################################


def mrqa_predictor(FLAGS, json_data):
  """
  Get prediction with the data got fron mrqa official request.
  """
  tf.logging.set_verbosity(tf.logging.INFO)

  sp_model = spm.SentencePieceProcessor()
  sp_model.Load(FLAGS.spiece_model_file)

  ### TPU Configuration
  run_config = model_utils.configure_tpu(FLAGS)

  model_fn = get_model_fn()  # model configs, load the trained model
  spm_basename = _get_spm_basename()

  tf.logging.info("Got Data from Server...")
  eval_data = arrange_mrqa_data(json_data, is_training=False)

  eval_writer = FeatureWriter(is_training=False)
  eval_features = []
  eval_features_inp = []

  def append_feature(feature):
    eval_features.append(feature)
    eval_features_inp.append(eval_writer.process_feature(feature))

  convert_examples_to_features(
			examples=eval_data,
			sp_model=sp_model,
			max_seq_length=FLAGS.max_seq_length,
			doc_stride=FLAGS.doc_stride,
			max_query_length=FLAGS.max_query_length,
			is_training=False,
			output_fn=append_feature) 

  predict_fn = tf.contrib.predictor.from_saved_model(FLAGS.export_dir_base)

  cur_results = []

  for num, eval_feature in enumerate(eval_features_inp):
    result = predict_fn({"examples":[eval_feature]})

    if len(cur_results) % 1000 == 0:
      tf.logging.info("Processing example: %d" % (len(cur_results)))

    unique_id = int(result["unique_ids"])
    start_top_log_probs = (
				[float(x) for x in result["start_top_log_probs"].flat])
    start_top_index = [int(x) for x in result["start_top_index"].flat]
    end_top_log_probs = (
				[float(x) for x in result["end_top_log_probs"].flat])
    end_top_index = [int(x) for x in result["end_top_index"].flat]

    cls_logits = float(result["cls_logits"].flat[0])

    cur_results.append(
				RawResult(
						unique_id=unique_id,
						start_top_log_probs=start_top_log_probs,
						start_top_index=start_top_index,
						end_top_log_probs=end_top_log_probs,
						end_top_index=end_top_index,
						cls_logits=cls_logits))

  ret = get_predictions(eval_data, eval_features, cur_results,
                        FLAGS.n_best_size, FLAGS.max_answer_length)
  print(ret)   ################################################################################### Print the output of the final result ############################################################
  return dict(ret)
################################################# ######################## after read_mrqa_data, model come here to predictor ###################################################################################################################
################################################# predictor ###################################################################################################################
def mrqa_tester():
  """
  Get prediction with the data got fron mrqa official request.
  """
  tf.logging.set_verbosity(tf.logging.INFO)

  sp_model = spm.SentencePieceProcessor()
  sp_model.Load(FLAGS.spiece_model_file)

  ### TPU Configuration
  run_config = model_utils.configure_tpu(FLAGS)

  model_fn = get_model_fn()  # model configs, load the trained model
  spm_basename = _get_spm_basename()

  dev_file = FLAGS.dev_dir+"/DROP-piece.jsonl" ########################################## The Input JSON
  tf.logging.info("Read data from {}".format(dev_file))
  eval_data = read_mrqa_data(dev_file, is_training=False)

  eval_writer = FeatureWriter(is_training=False)
  eval_features = []
  eval_features_inp = []

  def append_feature(feature):
    eval_features.append(feature)
    eval_features_inp.append(eval_writer.process_feature(feature))

  convert_examples_to_features(
			examples=eval_data,
			sp_model=sp_model,
			max_seq_length=FLAGS.max_seq_length,
			doc_stride=FLAGS.doc_stride,
			max_query_length=FLAGS.max_query_length,
			is_training=False,
			output_fn=append_feature) 

  predict_fn = tf.contrib.predictor.from_saved_model(FLAGS.export_dir_base)

  cur_results = []
  all__ = []
  listt = all__
  N = 0
  for num, eval_feature in enumerate(eval_features_inp):
    result = predict_fn({"examples":[eval_feature]})

    if len(cur_results) % 1000 == 0:
      tf.logging.info("Processing example: %d" % (len(cur_results)))

    unique_id = int(result["unique_ids"])
    start_top_log_probs = (
				[float(x) for x in result["start_top_log_probs"].flat])
    start_top_index = [int(x) for x in result["start_top_index"].flat]
    end_top_log_probs = (
				[float(x) for x in result["end_top_log_probs"].flat])
    end_top_index = [int(x) for x in result["end_top_index"].flat]

    cls_logits = float(result["cls_logits"].flat[0])

    cur_results.append(
				RawResult(
						unique_id=unique_id,
						start_top_log_probs=start_top_log_probs,
						start_top_index=start_top_index,
						end_top_log_probs=end_top_log_probs,
						end_top_index=end_top_index,
						cls_logits=cls_logits))

  ret, aaa = get_predictions(eval_data, eval_features, cur_results,
                        FLAGS.n_best_size, FLAGS.max_answer_length, listt)
  print(aaa)
  # all__.append(aaa)

  return dict(ret), aaa                                                         ####### Return the answers here ########



##########################################################################################FINAL RESULT PRINT HERE############################################################################################################
# if __name__ == "__main__":

#   # del_all_flags(FLAGS)
#   # FLAGS(sys.argv)
#   mrqa_tester()


In [ ]:
# dict with qid and top answers
# tf.app.flags.DEFINE_string('f', '', 'kernel') # for first time keep this (ig only in the colab), then comment this line(to remove dubplicate f).
reader_out, hmm = mrqa_tester()

INFO:tensorflow:Use MirroredStrategy with 8 devices.
INFO:tensorflow:Initializing RunConfig with distribution strategies.
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Read data from /content/data/dev/DROP-piece.jsonl
INFO:tensorflow:Converting 0/10 pos 0 neg 0
INFO:tensorflow:Total number of instances: 19 = pos 19 neg 0
INFO:tensorflow:Restoring parameters from /content/gdrive/MyDrive/HLTC-MRQA/exported-tf-model-1.15.2/variables/variables
INFO:tensorflow:Processing example: 0
INFO:tensorflow:Getting predictions
[[['Zoom?', -9.047911822795868], ["Zoom? So I've heard that's there for a long time", -11.476564645767212], ['meat', -11.548356056213379], ['meat and they announced the neat bar and the neat board which is a touch display and then Paul Lee', -12.239153861999512], ['meat and they announced the neat bar and the neat board', -12.789748191833496], ['Zoom', -12.792715072631836], ["meat and they announced the neat bar and the neat board which is a touch display an

In [ ]:
w  = [4,12,43.3,19,100]

In [ ]:
print(max(w))

100


In [ ]:
# Answer 1
s = ['90%', 'carbon footprint', 'extra', 'carbon footprint', '80%', 'about 90%', 'carbon', '20%', 'extra cost', 'about 90% of use cases', 'like an extra cost.', 'an extra cost', 'an extra cost.', 'like an extra cost', 'cost conscious', '90% of use cases', 'footprint', 'scheduling', 'extra cost.', 'carbon', 'drive a video', '20% and I would hear my interpreter voice at 80%', 'drive a video first experience', '$50', 'they were built from the ground up all the way down to the OS', 'like an extra', 'an extra', 'reduction', 'video', 'g', 'collaboration and communication', 'collaboration and communication should be seamless, even when the needs for collaboration communication', "add participants. So it's added to their calendar and there's a I'm just going to Breeze through it and then a visitor it preserves the room", 'collaboration', 'about 90% of', 'she is a bit of a rock star when it comes to being able to get things done', 'she is a bit of a rock star', 'communication', '$50 gift card', "add participants. So it's added to their calendar", 'native control', 'reloading', 'drive', 'they were built from the ground up', 'focus room', 'okay on time', 'focus', 'success is really going to be tied to our customer success', 'video first experience', '90% of', 'cost', "corporate background image, but I imagine that probably comes soon. \n\n- (26:02)  SPEAKER: Esther Yoon\nYeah, I wouldn't get out for sure because it's something that people have asked for that, you know branding is a huge part of large Enterprises", 'reduction within the carbon footprint', 'zoom platform', 'she is a bit of a rock star when it comes to being able to get things done being able to answer questions.', 'conference room Solutions', 'drive a video first', "success is inherently tied together, right? And so with that, you know, it's been a pleasure to work with, you know, the Dillman folks the building family and also Bobby and the rest of the team we've done a lot of really cool programs such as strategic. Eating programs you guys have built a lot of really bespoke capabilities for us on your guys's platform so that we can actually deliver happiness to our customers", "success is inherently tied together, right? And so with that, you know, it's been a pleasure to work with, you know, the Dillman folks the building family and also Bobby and the rest of the team we've done a lot of really cool programs such as strategic. Eating programs you guys have built a lot of really bespoke capabilities for us on your guys's platform so that we can actually deliver happiness", 'success', 'drive a video first experience,', 'success', 'zoom', 'collaboration and communication should be seamless,', "add participants. So it's added to their calendar and there's a I'm just going to Breeze through it and then a visitor it preserves the room and then you're golden.", '20% and I would hear my interpreter voice', 'native control integration. How much can the going to be customized', 'they were built from the ground up all the way down', 'corporate', 'IP', 'collaboration and communication should be seamless, even when the needs for collaboration communication and change in real time.', 'corporate background', 'reduction within', 'they were built from the ground up all the way down to the OS so their Tech', 'My pleasure', 'about', 'IP Control', 'reduction within the carbon', 'My pleasure.', 'conference room', 'enterprise-grade video conferencing without Enterprise size', 'native control integration. How much can the going to be customized? Okay, we talked about that you are going to be using IP Control', 'video first', 'scheduling display,', 'native control integration. How much can the going to be customized? Okay, we talked about that you are going to be using IP', 'enterprise-grade video conferencing', 'add participants.', "enterprise-grade video conferencing without Enterprise size complications. These are for people who don't want to do. Thinking they don't want to add multiple cameras. They're just like give me a box and I just wanted to plug it in real quick. And I want to do a hundred of these at skill", 'native', 'success is inherently tied together', 'communication and change', 'g a so if you guys have scheduling display,', 'zoom platform is a whole', 'ability to scale to different use cases.', "add participants. So it's added to their calendar and there's a I'm just going to Breeze through it and then a visitor", 'zoom', 'My', 'g a', "about 90% of use cases from what I've seen.", 'zoom rooms', 'like an', 'success is really going to be tied to our customer', 'okay', 'she is a bit of a rock star when it comes to being able to get things', 'reloading Keys', "IP Control. So that will be being bailed out more and more. Correct. \n\n- (27:28)  SPEAKER: Esther Yoon\nEverything is IP based as long as they can I pee command. Yep. All right kids Optical device. Yep. \n\n- (27:36)  SPEAKER: Melissa Dillman\nSo that'll be great. One of the other things that came out was the simultaneous interpretation", 'scheduling display, I would recommend testing it out you upload the map as a jpeg. Or PNG directly through the zoom', 'g a so if you guys have scheduling', 'she is a bit of a rock star when it comes', 'conference room Solutions joining us today coming off the heels of zootopia as well, which is kind of a big deal in the zoom', 'like an extra cost. Now. We have zoom rooms native Native room control integration.', 'scheduling display, I would recommend testing it out you upload the map', 'video first experience,', 'reduction within the', '$50 gift', '20% and I would hear my interpreter', "90% of use cases from what I've seen.", 'conference room Solutions joining us today coming off the heels of zootopia as well, which is kind of a big deal in the zoom World shuts down, San Jose. Pretty good.', 'conference room Solutions joining us today coming off the heels of zootopia as well, which is kind of a big deal in the zoom World shuts down, San Jose. Pretty good. What we really wanted to do was just have a little pet about coming up the hill so that show on what is in the future for the next coming up on Zoom.', 'reloading Keys.', "zoom platform is a whole and I wanted to thank we've got Esther Yoon who is the product marketing manager for conference room Solutions joining us today coming off the heels of zootopia as well, which is kind of a big deal in the zoom", 'success is inherently', 'an', 'scheduling display, I would recommend testing it out you upload the map as a jpeg. Or PNG directly through the zoom admin portal.', 'an extra cost. Now. We have zoom rooms native Native room control integration.', 'extra cost. Now. We have zoom rooms native Native room control integration.', 'carbon footprint because we can do everything via video.', 'carbon footprint hmm in talking about that reduction within the carbon footprint because we can do everything via video.', 'carbon footprint hmm in talking about that reduction within the carbon footprint', 'footprint because we can do everything via video.', 'carbon footprint hmm in talking about that reduction']

In [ ]:
s = [PrelimPrediction(feature_index=0, start_index=32, end_index=32, start_log_prob=-9.368597984313965, end_log_prob=-0.3939220607280731), PrelimPrediction(feature_index=0, start_index=240, end_index=241, start_log_prob=-9.159207344055176, end_log_prob=-1.1434053182601929), PrelimPrediction(feature_index=1, start_index=404, end_index=404, start_log_prob=-9.20814323425293, end_log_prob=-1.3890942335128784), PrelimPrediction(feature_index=1, start_index=368, end_index=369, start_log_prob=-9.735785484313965, end_log_prob=-0.9803922176361084), PrelimPrediction(feature_index=0, start_index=240, end_index=249, start_log_prob=-9.159207344055176, end_log_prob=-2.2522521018981934), PrelimPrediction(feature_index=0, start_index=240, end_index=253, start_log_prob=-9.159207344055176, end_log_prob=-2.268369197845459), PrelimPrediction(feature_index=1, start_index=368, end_index=368, start_log_prob=-9.735785484313965, end_log_prob=-1.779595136642456), PrelimPrediction(feature_index=1, start_index=368, end_index=377, start_log_prob=-9.735785484313965, end_log_prob=-1.868096113204956), PrelimPrediction(feature_index=0, start_index=240, end_index=240, start_log_prob=-9.159207344055176, end_log_prob=-2.5244574546813965), PrelimPrediction(feature_index=0, start_index=240, end_index=255, start_log_prob=-9.159207344055176, end_log_prob=-2.6515278816223145), PrelimPrediction(feature_index=1, start_index=184, end_index=185, start_log_prob=-9.782259941101074, end_log_prob=-2.59633207321167), PrelimPrediction(feature_index=0, start_index=32, end_index=44, start_log_prob=-9.368597984313965, end_log_prob=-3.2390778064727783), PrelimPrediction(feature_index=1, start_index=368, end_index=386, start_log_prob=-9.735785484313965, end_log_prob=-2.9417340755462646), PrelimPrediction(feature_index=0, start_index=32, end_index=36, start_log_prob=-9.368597984313965, end_log_prob=-3.5748379230499268), PrelimPrediction(feature_index=0, start_index=32, end_index=62, start_log_prob=-9.368597984313965, end_log_prob=-3.6320126056671143)]

In [ ]:
PrelimPrediction(feature_index=0, start_index=32, end_index=32, start_log_prob=-9.368597984313965, end_log_prob=-0.3939220607280731)

In [ ]:
ss = [['awesome', -9.762520045042038], ['consumer experience', -10.302612662315369], ['Melissa', -10.597237467765808], ['IP based', -10.716177701950073], ['consumer experience just like you would set up a display', -11.41145944595337], ['consumer experience just like you would set up a display with a sound system', -11.427576541900635], ['IP', -11.515380620956421], ['IP based as long as they can I pee', -11.603881597518921], ['consumer', -11.683664798736572], ['consumer experience just like you would set up a display with a sound system at home', -11.81073522567749], ['firmware', -12.378592014312744], ['awesome you covered a lot but you know what you left out.', -12.607675790786743], ['IP based as long as they can I pee command. Yep. All right kids Optical', -12.67751955986023], ['awesome you covered a lot', -12.943435907363892], ['awesome you covered a lot but you know what you left out. - (20:13)  SPEAKER: Esther Yoon\nWhat the phone', -13.000610589981079]]

In [ ]:
for item in ss:
  print(item)

['awesome', -9.762520045042038]
['consumer experience', -10.302612662315369]
['Melissa', -10.597237467765808]
['IP based', -10.716177701950073]
['consumer experience just like you would set up a display', -11.41145944595337]
['consumer experience just like you would set up a display with a sound system', -11.427576541900635]
['IP', -11.515380620956421]
['IP based as long as they can I pee', -11.603881597518921]
['consumer', -11.683664798736572]
['consumer experience just like you would set up a display with a sound system at home', -11.81073522567749]
['firmware', -12.378592014312744]
['awesome you covered a lot but you know what you left out.', -12.607675790786743]
['IP based as long as they can I pee command. Yep. All right kids Optical', -12.67751955986023]
['awesome you covered a lot', -12.943435907363892]
['awesome you covered a lot but you know what you left out. - (20:13)  SPEAKER: Esther Yoon\nWhat the phone', -13.000610589981079]


In [ ]:
sss = [['Lashawn Jackson.', -0.0833195741288364], ['Lashawn', -5.618622794747353], ['Lashawn Jackson', -5.979353919625282], ['Jackson.', -6.4887288408353925], ['shawn Jackson.', -7.439827188849449], ['It is Lashawn Jackson.', -7.570564657449722], ['La', -10.366392627358437], ['Lashawn Jackson. Thank you very much.', -10.664638057351112], ['It is Lashawn', -11.249517917633057], ['It is Lashawn Jackson', -11.37743854522705], ['Jackson', -11.518480777740479], ['It is Lashawn Jackson. Thank you very much.', -11.99608564376831], ['It is La', -12.521013259887695], ['shawn', -12.77404499053955], ['shawn Jackson', -13.511877536773682], ['shaw', -18.506267070770264]]

In [ ]:
for item in sss:
  print(item)

['Lashawn Jackson.', -0.0833195741288364]
['Lashawn', -5.618622794747353]
['Lashawn Jackson', -5.979353919625282]
['Jackson.', -6.4887288408353925]
['shawn Jackson.', -7.439827188849449]
['It is Lashawn Jackson.', -7.570564657449722]
['La', -10.366392627358437]
['Lashawn Jackson. Thank you very much.', -10.664638057351112]
['It is Lashawn', -11.249517917633057]
['It is Lashawn Jackson', -11.37743854522705]
['Jackson', -11.518480777740479]
['It is Lashawn Jackson. Thank you very much.', -11.99608564376831]
['It is La', -12.521013259887695]
['shawn', -12.77404499053955]
['shawn Jackson', -13.511877536773682]
['shaw', -18.506267070770264]


In [ ]:
# Answer 2
s = ['awesome', 'consumer experience', 'Melissa', 'IP based', 'consumer experience just like you would set up a display', 'consumer experience just like you would set up a display with a sound system', 'IP', 'IP based as long as they can I pee', 'consumer', 'consumer experience just like you would set up a display with a sound system at home', 'firmware', 'awesome you covered a lot but you know what you left out.', 'IP based as long as they can I pee command. Yep. All right kids Optical', 'awesome you covered a lot', 'awesome you covered a lot but you know what you left out. - (20:13)  SPEAKER: Esther Yoon\nWhat the phone']

[[90, 97], [784, 803], [2019, 2026], [1914, 1922], [784, 840], [784, 860], [1914, 1916], [1914, 1948], [784, 792], [784, 868], [1156, 1164], [90, 147], [1914, 1985], [90, 115], [90, 194]]

[[[90, 2026], "awesome you covered a lot but you know what you left out. - (20:13)  SPEAKER: Esther Yoon\nWhat the phone ah, okay. - (20:17)  SPEAKER: Melissa Dillman\nJust watched your video. - (20:19)  SPEAKER: Esther Yoon\nAgain, and how cool you were with the phone? - (20:22)  SPEAKER: Melissa Dillman\nThat's hilarious and I have to say we just so zootopia last year. We saw the phone, right? We saw some come this year. We implemented it. And yeah, obviously, I'm pretty I'm pretty deep with zoom. I use it everyday. I was a little concerned. Like, how do I have a call and how do I get it to a meeting and I have to say it's amazingly intuitive just like everything else was in right? It's kind of like a consumer experience just like you would set up a display with a sound system at home. And they had a video like Polly had a video where they were showing the insulation. It literally took two minutes. And then enhanced management, I already covered this but you know through the zoom admin portal click update and it does the entire software stack even down to the device firmware. So in a nutshell the way we're starting to structure. Our Zoom rooms is is really, you know, first and foremost it comes down to what does the customer want if they want that flexibility in their Huddle's basis and conference rooms. We're going to recommend them a Windows iot computer Mac or something like that so they can have that bespoke experience if they want that. So I watched a bunch of them when I came home as you know to get a chance to watch it again or enjoy it again. So it's awesome some rooms native control integration. How much can the going to be customized? Okay, we talked about that you are going to be using IP Control. So that will be being bailed out more and more. Correct. - (27:28)  SPEAKER: Esther Yoon\nEverything is IP based as long as they can I pee command. Yep. All right kids Optical device. Yep. - (27:36)  SPEAKER: Melissa"]]


In [ ]:
for item in s:
  print(item)

awesome
consumer experience
Melissa
IP based
consumer experience just like you would set up a display
consumer experience just like you would set up a display with a sound system
IP
IP based as long as they can I pee
consumer
consumer experience just like you would set up a display with a sound system at home
firmware
awesome you covered a lot but you know what you left out.
IP based as long as they can I pee command. Yep. All right kids Optical
awesome you covered a lot
awesome you covered a lot but you know what you left out. - (20:13)  SPEAKER: Esther Yoon
What the phone


In [ ]:
for item in hmm:
  print(item)

[['Zoom?', -9.047911822795868], ["Zoom? So I've heard that's there for a long time", -11.476564645767212], ['meat', -11.548356056213379], ['meat and they announced the neat bar and the neat board which is a touch display and then Paul Lee', -12.239153861999512], ['meat and they announced the neat bar and the neat board', -12.789748191833496], ['Zoom', -12.792715072631836], ["meat and they announced the neat bar and the neat board which is a touch display and then Paul Lee announced. Their Appliance x-series. So there is an x 30 which is for smaller rooms. And then there's an x 54 larger rooms", -13.159703254699707], ["meat and they announced the neat bar and the neat board which is a touch display and then Paul Lee announced. Their Appliance x-series. So there is an x 30 which is for smaller rooms. And then there's an x 54", -13.182027816772461]]
[['Not much text', -6.257963299751282], ['Not much text absolutely', -7.6872111558914185], ['Not much text absolutely.', -8.112068176269531],

In [ ]:
# get the qid and get the respective answer
reader_ans = reader_out['customer_1']

# Step 3: THE GENERATOR

In [ ]:
# get actual content for the top candidates
paragraphs = list(reader_out.values())
meta_data = [{"document_id": 'customer_1', "paragraph_id": None}]

documents = []
for para, meta in zip(paragraphs, meta_data):
    documents.append(
        Document(
            id=meta["document_id"],
            text=para,
            meta=meta.get("meta", {})
        ))

In [ ]:
documents

[{'text': 'Necker Island', 'score': None, 'probability': None, 'question': None, 'meta': {}, 'embedding': None, 'id': 'customer_1'}]

In [ ]:
# Initialize RAG Generator
generator = RAGenerator(
    model_name_or_path="facebook/rag-token-nq",
    use_gpu=True,
    top_k=1,
    max_length=200,
    min_length=2,
    embed_title=False,
    num_beams=5,
)

06/17/2021 11:49:56 - INFO - filelock -   Lock 140010000895504 acquired on /root/.cache/huggingface/transformers/6337b0203e20d15c98f5e500e1e673c74e71bb8617b2753a53663b9b8e6dfc1a.59948e1fef260da10a0cecb8b6862373c32f40001848a63f985ab4f9d787f3f1.lock


06/17/2021 11:49:57 - INFO - filelock -   Lock 140010000895504 released on /root/.cache/huggingface/transformers/6337b0203e20d15c98f5e500e1e673c74e71bb8617b2753a53663b9b8e6dfc1a.59948e1fef260da10a0cecb8b6862373c32f40001848a63f985ab4f9d787f3f1.lock


/usr/local/lib/python3.7/dist-packages/transformers/models/bart/configuration_bart.py:178: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions.The config can simply be saved and uploaded again to be fixed.
  f"Please make sure the config includes `forced_bos_token_id={self.bos_token_id}` in future versions."
06/17/2021 11:49:57 - INFO - filelock -   Lock 140009993585680 acquired on /root/.cache/huggingface/transformers/26cf899a0974235af1f84469ddd94d2ee83c803c23ecead93b511ce8a0744f5c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


06/17/2021 11:49:58 - INFO - filelock -   Lock 140009993585680 released on /root/.cache/huggingface/transformers/26cf899a0974235af1f84469ddd94d2ee83c803c23ecead93b511ce8a0744f5c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


06/17/2021 11:49:58 - INFO - filelock -   Lock 140009993732176 acquired on /root/.cache/huggingface/transformers/d4df3c917efc1bf4cde9515ac4432cddf040d3ddacfad55c85445bc985f58ceb.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d.lock


...

06/17/2021 11:50:02 - INFO - filelock -   Lock 140009993743824 acquired on /root/.cache/huggingface/transformers/a2f0711fd4c199beaf3ff2626c4a723e107e79a8d065099a3d24e7aa361c0f18.67d01b18f2079bd75eac0b2f2e7235768c7f26bd728e7a855a1c5acae01a91a8.lock


06/17/2021 11:50:03 - INFO - filelock -   Lock 140009993743824 released on /root/.cache/huggingface/transformers/a2f0711fd4c199beaf3ff2626c4a723e107e79a8d065099a3d24e7aa361c0f18.67d01b18f2079bd75eac0b2f2e7235768c7f26bd728e7a855a1c5acae01a91a8.lock


06/17/2021 11:50:04 - INFO - filelock -   Lock 140009845683280 acquired on /root/.cache/huggingface/transformers/6288b70ae87b5989f6801120b25a2f6ccf2e88c2f622d94805717b5b043ff71b.bceb1ea6e95a1b39d0a717f315800a39b0847e5cdeeb42615fb498775f632ee6.lock


06/17/2021 11:50:45 - INFO - filelock -   Lock 140009845683280 released on /root/.cache/huggingface/transformers/6288b70ae87b5989f6801120b25a2f6ccf2e88c2f622d94805717b5b043ff71b.bceb1ea6e95a1b39d0a717f315800a39b0847e5cdeeb42615fb498775f632ee6.lock


Some weights of RagTokenForGeneration were not initialized from the model checkpoint at facebook/rag-token-nq and are newly initialized: ['rag.generator.lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Now generate answer from question and retrieved documents
predicted_result = generator.predict(
    query="From where Richard Branson was supposed to call?",
    documents=documents,
    top_k=1
)

AttributeError: ignored